In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from pipeline import AnomalyDetectionPipeline, load_model, plot_lob_snapshot
import preprocessing as prep
import machine_learning as ml
import config

In [2]:
def plot_feature_attribution(attributions, feature_names, title="Feature Importance", top_k=20):
    """
    Aggregates attribution over time (sum or mean) and plots top k features.
    Attributions shape: (1, Seq_Len, Feat)
    """
    # Sum absolute attributions over time
    # We want to know which feature contributed most overall
    attr_sum = np.sum(np.abs(attributions), axis=(0, 1))
    
    # Create DataFrame
    df_attr = pd.DataFrame({
        'Feature': feature_names,
        'Attribution': attr_sum
    }).sort_values(by='Attribution', ascending=False).head(top_k)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_attr, x='Attribution', y='Feature', hue='Feature', palette='viridis', legend=False)
    plt.title(title)
    plt.xlabel("Integrated Gradients Magnitude")
    plt.tight_layout()
    plt.show()
    return df_attr

In [3]:
# Select model
DATASET = 'TOTF'
MODEL = 'prae'
SCALER = 'box-cox'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {DEVICE}")

config_path = f"models/{DATASET}_{SCALER}_{MODEL}_config.json"
weights_path = f"models/{DATASET}_{SCALER}_{MODEL}_weights.pth"
scaler_path = f"models/{DATASET}_{SCALER}_{MODEL}_scaler.pkl"
conf = config.DATASETS[DATASET]

Running on cuda


In [4]:
# Initialize pipeline
setup_pipeline = AnomalyDetectionPipeline()

# Load data
if DATASET == 'TOTF': setup_pipeline.load_data(conf['path'])
elif DATASET == 'LOBSTER': setup_pipeline.raw_df = prep.load_lobster_data(conf['orderbook'], conf['message'])

# Engineer features
setup_pipeline.engineer_features(feature_sets=['base', 'tao', 'hawkes', 'poutre', 'ofi'])

# Scale and sequence data
setup_pipeline.scale_and_sequence(method='box-cox', train_ratio=0.7)

# Match training logic for test set
test_start_idx = int(len(setup_pipeline.processed_df) * 0.85)
test_df = setup_pipeline.processed_df.iloc[test_start_idx:].reset_index(drop=True)

Pipeline initialized on device: cuda
Loading data from data/TOTF.PA-book/2015-01-02-TOTF.PA-book.csv.gz...
Successfully loaded 640429 rows.
Engineering features: ['base', 'tao', 'hawkes', 'poutre', 'ofi']...
Feature Engineering complete. Total features: 130
Preprocessing with method: box-cox...
Dropping 2 constant/zero-variance features: ['ask_sweep_cost', 'ask-volume-10']
Data split: Train 448300, Val 96064, Test 96065


In [5]:
# Load model
pipeline, cfg = load_model(config_path, test_df, setup_pipeline.feature_names)

state_dict = torch.load(weights_path, map_location=pipeline.device)

Pipeline initialized on device: cuda


In [6]:
anomaly_scores = pipeline.predict(pipeline.X_test)
threshold = np.percentile(anomaly_scores, 99)
anomaly_indices = np.where(anomaly_scores > threshold)[0]

In [7]:
target_idx = anomaly_indices[0]
print(f"Analyzing Anomaly at Index: {target_idx}")

raw_seq = pipeline.X_test[target_idx][0]
x_seq = torch.tensor(raw_seq, dtype=torch.float32).unsqueeze(0).to(pipeline.device)
print(f"x_seq shape: {x_seq.shape}")

Analyzing Anomaly at Index: 94933
x_seq shape: torch.Size([1, 25, 128])


C:\Users\adoni\AppData\Local\Temp\ipykernel_5176\2774499043.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_seq = torch.tensor(raw_seq, dtype=torch.float32).unsqueeze(0).to(pipeline.device)


In [8]:
class FlattenWrapper(torch.nn.Module):
    """
    Wraps PNN to accept (Batch, Seq, Feat) input by flattening it internally.
    """
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, x):
        # x shape: (Batch, Seq, Feat)
        # Flatten to: (Batch, Seq * Feat)
        batch_size = x.size(0)
        x_flat = x.reshape(batch_size, -1)
        return self.model(x_flat)

In [9]:
if pipeline.model_type == 'pnn':
    model_to_explain = FlattenWrapper(pipeline.model)
    target_fn = ml.pnn_uncertainty
else:
    model_to_explain = pipeline.model
    
    if pipeline.model_type == 'transformer_ocsvm': target_fn = ml.transformer_reconstruction_loss
    elif pipeline.model_type == 'prae': target_fn = ml.prae_anomaly_score_func

In [10]:
# Setup
IG = ml.IntegratedGradients(pipeline.model)
x_seq_tensor = torch.tensor(x_seq).unsqueeze(0).to(pipeline.device) # Shape: (1, 25, 14)

C:\Users\adoni\AppData\Local\Temp\ipykernel_5176\3114895945.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_seq_tensor = torch.tensor(x_seq).unsqueeze(0).to(pipeline.device) # Shape: (1, 25, 14)


In [11]:
# Run IG with selected target function
attrs = IG.attribute(
    inputs=x_seq_tensor,
    target_func=target_fn,
    n_steps=50
)

attrs_np = attrs.detach().cpu().numpy()

In [12]:
top_features = plot_feature_attribution(attrs_np, pipeline.feature_names, title=f"Feature Importance for {MODEL.upper()} on {DATASET}")

top_5_indices = [pipeline.feature_names.index(feat) for feat in top_features['Feature'].head(5).values]
top_5_names = top_features['Feature'].head(5).values

heatmap_data = attrs_np[0, :, top_5_indices]

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap="RdBu_r", center=0, yticklabels=top_5_names)
plt.title(f"Temporal Feature Attribution for Top 5 Features - {MODEL.upper()} on {DATASET}")
plt.xlabel("Time Steps (Sequence)")
plt.show()

plot_lob_snapshot(pipeline, target_idx)

ValueError: Per-column arrays must each be 1-dimensional